In [271]:
from manim import *
import numpy as np
from typing import List, TypeVar

quality = "ql"

In [272]:
# Font sizes
SMALL_FONT = 24
MED_FONT = 32
LARGE_FONT = 48

# Colors
C_HIGHLIGHT = TEAL_C
C_EMPHASIS = ORANGE

In [273]:
from math import acos, atan2, cos, sin

def squared_norm(vector: np.array) -> float:
    return np.dot(vector, vector)

def norm(vector: np.array) -> float:
    """
    Returns the length of a vector.
    """
    return squared_norm(vector) ** 0.5


def normalize(vector: np.array) -> np.array:
    """
    Returns the vector normalized so that it has length 1.
    """
    return vector / norm(vector)

def circle_to_circle_tangent(center1: np.array, radius1: float, center2: np.array, radius2: float) -> List[np.array]:
    """
    Returns the outer tangent line between two circles.
    The tangent is such that it is on the outside of 1 and 2 when they are arranged in counter-clockwise fashion.
    """
    radius1 *= -1
    radius2 *= -1
    dist = norm(center2 - center1)

    delta = (center2 - center1) / dist
    cross = np.array([-delta[1], delta[0], 0])
    alpha = (radius1 - radius2) / dist
    beta = (1 - alpha ** 2) ** 0.5
    return [
        center1 + (alpha * delta + beta * cross) * radius1,
        center2 + (alpha * delta + beta * cross) * radius2
    ]

def circle_to_point_tangent(center: np.array, radius: float, point: np.array) -> np.array:
    dist = norm(point - center)
    angle = acos(radius / dist)
    angle_offset = atan2(point[1] - center[1], point[0] - center[0])
    return np.array([
        center[0] + radius * cos(angle_offset - angle),
        center[1] + radius * sin(angle_offset - angle),
        0
    ])

def circle_to_point_tangent(point: np.array, center: np.array, radius: float) -> List[float]:
    return circle_to_circle_tangent(center, -radius, point)

In [274]:

T = TypeVar('T')
class PlateCircle(VGroup):
    def __init__(self, radius: float, offset: float) -> None:
        super().__init__()
        self._inner_circle = Circle(radius)
        self._outer_circle = Circle(radius + offset)
        self.add(self._inner_circle)
        self.add(self._outer_circle)

    def get_center(self) -> np.array:
        return self._inner_circle.get_center()
    
    def get_radius(self) -> float:
        return self._outer_circle.radius
    
    def copy(self: T, center: np.array) -> T:
        return super().copy().move_to(center)
    
    def draw_inner_circle(self) -> Animation:
        return GrowFromCenter(self._inner_circle)
    
    def draw_outer_circle(self) -> Animation:
        return GrowFromCenter(self._outer_circle)

    
    def draw_annulus(self) -> Animation:
        """
        Shades the annulus region between the two circles.
        """
        

In [275]:
class Plate(VGroup):
    def __init__(self, points: List[PlateCircle], inside_indices: List[int]) -> None:
        self._points = points
        super().__init__(*self._points)
        self._inside = [points[i] for i in inside_indices]
        self._boundary = filter(lambda p: p not in self._inside, self._points)
    
    def draw_inner_circles(self) -> AnimationGroup:
        return AnimationGroup(*[x.draw_inner_circle() for x in self._points], lag_ratio=0.35)

    def draw_outer_circles(self) -> AnimationGroup:
        return AnimationGroup(*[x.draw_outer_circle() for x in self._points], lag_ratio =0.35)
    
    # def draw_boundary(self) -> AnimationGroup:
    #     for i in range(len(self._boundary)):
    #         curr = self._boundary[i]
    #         next = self._boundary[(i + 1) % len(self._boundary)]

In [276]:
%%manim -v WARNING --disable_caching -$quality DrawPlateCirclesScene

class DrawPlateCirclesScene(Scene):
    def construct(self):
        smallBase = PlateCircle(0.15, 0.2)
        mediumBase = PlateCircle(0.4, 0.2)

        front_hole = np.array([-4, -2, 0])
        middle_hole = np.array([-1.5, 1.25, 0])
        back_hole = np.array([2.5, 2.5, 0])
        back_offset = np.array([0.75, 0.6, 0])

        points = [
            smallBase.copy(back_hole + back_offset),
            # smallBase.copy(back_hole + )
            mediumBase.copy(back_hole),
            smallBase.copy((middle_hole + back_hole) / 2),
            mediumBase.copy(middle_hole),
            smallBase.copy((front_hole + middle_hole) / 2),
            mediumBase.copy(front_hole),
        ]

        plate = Plate(points, [2, 4])

        self.play(plate.draw_inner_circles())
        self.wait(1)
        self.play(plate.draw_outer_circles())

        # left.draw_outer_circle()
        # right.draw_outer_circle()